<a href="https://colab.research.google.com/github/palmmywatch/data_science_bootcamp07/blob/main/Machine_Learning_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning in R
House Price India

Load Library

In [ ]:
install.packages("readxl")
install.packages("dplyr")
install.packages("tidyverse")
install.packages("caret")
install.packages("randomForest")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘textshaping’, ‘conflicted’, ‘jsonlite’, ‘ragg’, ‘stringr’


Warning message in install.packages("tidyverse"):
“installation of package ‘textshaping’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘ragg’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, 

In [ ]:
library(readxl)
library(dplyr)
library(tidyverse)
library(caret)
library(randomForest)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ stringr 1.5.0
✔ tidyr   1.3.0     ✔ forcats 1.0.0
✔ readr   2.1.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following 

Read Excel and Preview Data

In [ ]:
# read excel
house_price_india <- read_excel("House Price India.xlsx")

In [ ]:
# review datasets
house_price_india %>% 
  glimpse()

Rows: 14,620
Columns: 23
$ id                                      <dbl> 6762810145, 6762810635, 676281…
$ Date                                    <dbl> 42491, 42491, 42491, 42491, 42…
$ `number of bedrooms`                    <dbl> 5, 4, 5, 4, 3, 3, 5, 3, 3, 4, …
$ `number of bathrooms`                   <dbl> 2.50, 2.50, 2.75, 2.50, 2.00, …
$ `living area`                           <dbl> 3650, 2920, 2910, 3310, 2710, …
$ `lot area`                              <dbl> 9050, 4000, 9480, 42998, 4500,…
$ `number of floors`                      <dbl> 2.0, 1.5, 1.5, 2.0, 1.5, 1.0, …
$ `waterfront present`                    <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ `number of views`                       <dbl> 4, 0, 0, 0, 0, 0, 2, 0, 2, 0, …
$ `condition of the house`                <dbl> 5, 5, 3, 3, 4, 4, 3, 5, 4, 5, …
$ `grade of the house`                    <dbl> 10, 8, 8, 9, 8, 9, 10, 8, 8, 7…
$ `Area of the house(excluding basement)` <dbl> 3370, 1910, 2910, 3310, 1880, …
$ `Area of the 

Select Features

In [ ]:
# select column
house_india <- house_price_india %>% 
  select(bed = `number of bedrooms`,
         bath = `number of bathrooms`,
         living = `living area`,
         area = `lot area`,
         floors = `number of floors`,
         waterfront = `waterfront present`,
         grade = `grade of the house`,
         school = `Number of schools nearby`,
         airport = `Distance from the airport`,
         price = Price)

house_india %>%
  head(10)

bed,bath,living,area,floors,waterfront,grade,school,airport,price
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5,2.50,3650,9050,2.0,0,10,2,58,2380000
4,2.50,2920,4000,1.5,0,8,2,51,1400000
5,2.75,2910,9480,1.5,0,8,1,53,1200000
4,2.50,3310,42998,2.0,0,9,3,76,838000
3,2.00,2710,4500,1.5,0,8,1,51,805000
3,2.50,2600,4750,1.0,0,9,1,67,790000
5,3.25,3660,11995,2.0,0,10,3,72,785000
3,1.75,2240,10578,2.0,0,8,3,71,750000
3,2.50,2390,6550,1.0,0,8,1,73,750000


Split Data

In [ ]:
# function split data
spData <- function(dataset, trainRatio) {
  # random sampling
  set.seed(11)
  n <- nrow(dataset)
  id <-  sample(1:n, size = trainRatio * n)
  
  # split
  trainData <- dataset[id, ]
  testData <- dataset[-id, ]
  
  # return data
  list(train = trainData,
       test  = testData)
}

In [ ]:
# split data
set.seed(11)
split_data <- spData(house_india, 0.9)
train_data <- split_data$train 
test_data <- split_data$test

Train Model

In [ ]:
# k-fold = 5
ctrl <- trainControl(
  method = "cv",
  number = 5,
  verboseIter = TRUE
)

In [ ]:
# linear model
lm_model <- train(log(price) ~ bed + bath + living + area + floors + 
                               waterfront + grade + school + airport,
            data = train_data,
            method = "lm",
            trControl = ctrl)

+ Fold1: intercept=TRUE 
- Fold1: intercept=TRUE 
+ Fold2: intercept=TRUE 
- Fold2: intercept=TRUE 
+ Fold3: intercept=TRUE 
- Fold3: intercept=TRUE 
+ Fold4: intercept=TRUE 
- Fold4: intercept=TRUE 
+ Fold5: intercept=TRUE 
- Fold5: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [ ]:
lm_model

Linear Regression 

13158 samples
    9 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 10527, 10525, 10527, 10528, 10525 
Resampling results:

  RMSE       Rsquared   MAE      
  0.3406481  0.5818774  0.2750971

Tuning parameter 'intercept' was held constant at a value of TRUE

In [ ]:
# random forest
rf_model <- train(log(price) ~ bed + bath + living + area + floors + 
                    waterfront + grade + school + airport,
                  data = train_data,
                  method = "rf",
                  trControl = ctrl)

+ Fold1: mtry=2 
- Fold1: mtry=2 
+ Fold1: mtry=5 
- Fold1: mtry=5 
+ Fold1: mtry=9 
- Fold1: mtry=9 
+ Fold2: mtry=2 
- Fold2: mtry=2 
+ Fold2: mtry=5 
- Fold2: mtry=5 
+ Fold2: mtry=9 
- Fold2: mtry=9 
+ Fold3: mtry=2 
- Fold3: mtry=2 
+ Fold3: mtry=5 
- Fold3: mtry=5 
+ Fold3: mtry=9 
- Fold3: mtry=9 
+ Fold4: mtry=2 
- Fold4: mtry=2 
+ Fold4: mtry=5 
- Fold4: mtry=5 
+ Fold4: mtry=9 
- Fold4: mtry=9 
+ Fold5: mtry=2 
- Fold5: mtry=2 
+ Fold5: mtry=5 
- Fold5: mtry=5 
+ Fold5: mtry=9 
- Fold5: mtry=9 
Aggregating results
Selecting tuning parameters
Fitting mtry = 2 on full training set


In [ ]:
rf_model

Random Forest 

13158 samples
    9 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 10526, 10526, 10527, 10526, 10527 
Resampling results across tuning parameters:

  mtry  RMSE       Rsquared   MAE      
  2     0.3153199  0.6435751  0.2509028
  5     0.3176214  0.6367633  0.2509350
  9     0.3202903  0.6310204  0.2524052

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 2.

In [ ]:
# KNN
knn_model <- train(log(price) ~ bed + bath + living + area + floors + 
                    waterfront + grade + school + airport,
                  data = train_data,
                  method = "knn",
                  trControl = ctrl)

+ Fold1: k=5 
- Fold1: k=5 
+ Fold1: k=7 
- Fold1: k=7 
+ Fold1: k=9 
- Fold1: k=9 
+ Fold2: k=5 
- Fold2: k=5 
+ Fold2: k=7 
- Fold2: k=7 
+ Fold2: k=9 
- Fold2: k=9 
+ Fold3: k=5 
- Fold3: k=5 
+ Fold3: k=7 
- Fold3: k=7 
+ Fold3: k=9 
- Fold3: k=9 
+ Fold4: k=5 
- Fold4: k=5 
+ Fold4: k=7 
- Fold4: k=7 
+ Fold4: k=9 
- Fold4: k=9 
+ Fold5: k=5 
- Fold5: k=5 
+ Fold5: k=7 
- Fold5: k=7 
+ Fold5: k=9 
- Fold5: k=9 
Aggregating results
Selecting tuning parameters
Fitting k = 9 on full training set


In [ ]:
knn_model

k-Nearest Neighbors 

13158 samples
    9 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 10528, 10527, 10526, 10525, 10526 
Resampling results across tuning parameters:

  k  RMSE       Rsquared   MAE      
  5  0.3909401  0.4581381  0.3094861
  7  0.3825116  0.4763101  0.3036944
  9  0.3787771  0.4844797  0.3008964

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was k = 9.

Normolization

In [ ]:
test_data$price_norm <- log(test_data$price)

Score Model

In [ ]:
# linear
p_lm <- predict(lm_model, newdata = test_data)

# random forest
p_rf <- predict(rf_model, newdata = test_data)

# knn
p_knn <- predict(knn_model, newdata = test_data)

Evaluate Model

In [ ]:
# metric function
eMetric <- function(actual, prediction) {
  # mean absolute error
  abs_error <- abs(actual - prediction)
  mae_metric <- mean(abs_error)
  
  # mse
  sq_error <- (actual - prediction)**2
  mse_metric <- mean(sq_error)
  
  # rmse
  rmse_metric <- sqrt(mse_metric)
  
  # return
  list(mae_metric,
       mse_metric,
       rmse_metric)
}

In [ ]:
# linear
e_lm <- eMetric(test_data$price_norm, p_lm)

# random forest
e_rf <- eMetric(test_data$price_norm, p_rf)

# knn
e_knn <- eMetric(test_data$price_norm, p_knn)

Result

In [ ]:
lm_result <- function() {
  cat("MAE:", e_lm[[1]],
      "\nMSE:", e_lm[[2]],
      "\nRMSE:", e_lm[[3]])
}

rf_result <- function() {
  cat("MAE:", e_rf[[1]],
      "\nMSE:", e_rf[[2]],
      "\nRMSE:", e_rf[[3]])
}

knn_result <- function() {
  cat("MAE:", e_knn[[1]],
      "\nMSE:", e_knn[[2]],
      "\nRMSE:", e_knn[[3]])
}

In [ ]:
lm_result()

MAE: 0.2674759 
MSE: 0.1138669 
RMSE: 0.3374417

In [ ]:
rf_result()

MAE: 0.2442743 
MSE: 0.09582324 
RMSE: 0.3095533

In [ ]:
knn_result()

MAE: 0.3004687 
MSE: 0.1424063 
RMSE: 0.3773676